In [43]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from concurrent.futures import ProcessPoolExecutor
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, SGDRegressor
from functools import partial
from sklearn.model_selection import train_test_split

In [49]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder='passthrough')
xgb_pipe = Pipeline([
    ('onehot', onehot_cols),
    ('standardize', StandardScaler()),
    ('xgb', SGDRegressor())
])

In [50]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithm=xgb_pipe)
    return thesis


In [51]:
thesis = full_run()

Finished experiment! Elapsed time: 0.08477067947387695, total Elapsed time: 0.08477067947387695, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.051860809326171875, total Elapsed time: 0.13663148880004883, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.057843923568725586, total Elapsed time: 0.19547367095947266, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.061867475509643555, total Elapsed time: 0.2573411464691162, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished experiment! Elapsed time: 0.049833059310913086, total Elapsed time: 0.3071742057800293, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift 

In [47]:
results = thesis.resultsToDF()

In [48]:
results.groupby(["Drift type", "Drift time", "Drift magnitude"]).mean()

MSE      MAPE  \
Drift type        Drift time     Drift magnitude                         
Incremental Drift Fully observed Large             63.190248  0.035376   
                                 Small             12.995389  0.021229   
                  Half observed  Large            407.514901  0.093703   
                                 Small             32.083532  0.032412   
                  Unobserved     Large            336.018969  0.077412   
                                 Small             27.444849  0.029425   
No Drift          No Drift       No Drift           9.667202  0.019443   
Sudden Drift      Fully observed Large             71.991320  0.037548   
                                 Small             13.605258  0.021677   
                  Half observed  Large            109.000345  0.044584   
                                 Small             16.143691  0.023048   
                  Unobserved     Large            403.205269  0.105791   
                                 Small             34.647125  0.034615   

                                                      SMAPE  
Drift type        Drift time     Drift magnitude             
Incremental Drift Fully observed Large             3.434667  
                                 Small             2.108000  
                  Half observed  Large             9.631333  
                                 Small             3.265333  
                  Unobserved     Large             7.940000  
                                 Small             2.960000  
No Drift          No Drift       No Drift          1.940000  
Sudden Drift      Fully observed Large             3.630000  
                                 Small             2.149333  
                  Half observed  Large             4.661333  
                                 Small             2.321333  
                  Unobserved     Large            10.862667  
                                 Small             3.489333

In [13]:
results.sort_values("SMAPE", ascending=False)

,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,MAPE,SMAPE
159,"[0, 3, 6]",Sudden Drift,Large,Important,Unobserved,2433.100539,0.272090,31.56
48,0,Sudden Drift,Large,Important,Unobserved,2418.464965,0.271176,31.45
85,3,Sudden Drift,Large,Important,Unobserved,2330.424853,0.266029,30.76
122,6,Sudden Drift,Large,Important,Unobserved,2270.048255,0.262627,30.30
11,None,Sudden Drift,Large,Important,Unobserved,2222.020873,0.259942,29.95
...,...,...,...,...,...,...,...,...
138,6,Incremental Drift,Large,Important,Fully observed,20.147063,0.020016,2.02
83,3,Sudden Drift,Large,Important,Fully observed,19.242779,0.019737,2.00
101,3,Incremental Drift,Large,Important,Fully observed,18.895642,0.019498,1.97
120,6,Sudden Drift,Large,Important,Fully observed,19.285891,0.019430,1.96
